In [1]:
import d3rlpy
from d3rlpy.algos import COMBO
from sklearn.model_selection import train_test_split
import gym
import numpy as np
import encoders

## Params

In [2]:
d3rlpy.seed(1)
use_gpu = True
# prepare environment
env = gym.make("InvertedPendulum-v4")
eval_env = gym.make("InvertedPendulum-v4")
env.reset(seed=1)
eval_env.reset(seed=1)

/home/neelay/.local/share/virtualenvs/symmetry-cs285-MHAjodT-/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:69: UserWarning: WARN: Agent's minimum action space value is -infinity. This is probably too low.
  logger.warn(
/home/neelay/.local/share/virtualenvs/symmetry-cs285-MHAjodT-/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:73: UserWarning: WARN: Agent's maximum action space value is infinity. This is probably too high
  logger.warn(
/home/neelay/.local/share/virtualenvs/symmetry-cs285-MHAjodT-/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


array([ 0.00023643,  0.00900927, -0.00711681,  0.00897299])

## Create dataset

In [14]:
actor_encoder = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# setup algorithm
sac = d3rlpy.algos.SAC(
    batch_size=256,
    actor_encoder_factory=actor_encoder,
    actor_learning_rate=3e-4,
    critic_learning_rate=3e-4,
    temp_learning_rate=3e-4,
    use_gpu=use_gpu
)

# prepare utilities
buffer = d3rlpy.online.buffers.ReplayBuffer(maxlen=1000000, env=env)

# start training
sac.fit_online(
    env,
    buffer,
    eval_env=eval_env,
    n_steps=100000,
    n_steps_per_epoch=1000,
    update_interval=1,
    update_start_step=1000,
    tensorboard_dir='tensorboard_logs'
)

2023-10-01 15:22:21 [info     ] Directory is created at d3rlpy_logs/SAC_online_20231001152221
2023-10-01 15:22:21 [debug    ] Building model...
2023-10-01 15:22:21 [debug    ] Model has been built.
2023-10-01 15:22:21 [info     ] Parameters are saved to d3rlpy_logs/SAC_online_20231001152221/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'actor_learning_rate': 0.0003, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'gamma': 0.99, 'generated_maxlen': 100000, 'initial_temperature': 1.0, 'n_critics': 2, '

  0%|          | 0/100000 [00:00<?, ?it/s]

2023-10-01 15:22:23 [info     ] Model parameters are saved to d3rlpy_logs/SAC_online_20231001152221/model_1000.pt
2023-10-01 15:22:23 [info     ] SAC_online_20231001152221: epoch=1 step=1000 epoch=1 metrics={'time_inference': 0.0011901423931121827, 'time_environment_step': 6.751775741577148e-05, 'time_step': 0.0013019063472747804, 'rollout_return': 10.06060606060606, 'evaluation': 14.7} step=1000
2023-10-01 15:22:37 [info     ] Model parameters are saved to d3rlpy_logs/SAC_online_20231001152221/model_2000.pt
2023-10-01 15:22:37 [info     ] SAC_online_20231001152221: epoch=2 step=2000 epoch=2 metrics={'time_inference': 0.0012940735816955567, 'time_environment_step': 0.0001036219596862793, 'time_sample_batch': 0.000309492826461792, 'time_algorithm_update': 0.011871881484985351, 'temp_loss': 1.372351692557335, 'temp': 0.8709251883625985, 'critic_loss': 0.6754881671965122, 'actor_loss': -4.277053299486637, 'time_step': 0.013659589290618897, 'rollout_return': 17.75, 'evaluation': 102.4} ste

In [15]:
# export replay buffer as MDPDataset
dataset = buffer.to_mdp_dataset()

# save MDPDataset
dataset.dump('d3rlpy_data/inverted_pendulum2.h5')

## Load the dataset

In [3]:
dataset = d3rlpy.dataset.MDPDataset.load('d3rlpy_data/inverted_pendulum2.h5')

In [4]:
train_episodes, test_episodes = train_test_split(dataset)

## Dynamics learning

In [5]:
encoder_factory = encoders.InvertedPendulumEncoderFactory()
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=True, encoder_factory=encoder_factory)

Using InvertedPendulumEncoderFactory


In [7]:
# same as algorithms
dynamics.fit(train_episodes,
             eval_episodes=test_episodes,
             n_epochs=100,
             scorers={
                'observation_error': d3rlpy.metrics.scorer.dynamics_observation_prediction_error_scorer,
                'reward_error': d3rlpy.metrics.scorer.dynamics_reward_prediction_error_scorer,
                'variance': d3rlpy.metrics.scorer.dynamics_prediction_variance_scorer,
             },
            tensorboard_dir='tensorboard_logs/dynamics')

2023-10-02 22:22:59 [debug    ] RoundIterator is selected.
2023-10-02 22:22:59 [info     ] Directory is created at d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002222259
2023-10-02 22:22:59 [warning  ] Skip building models since they're already built.
2023-10-02 22:22:59 [info     ] Parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002222259/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'encoder_factory': {'type': 'none', 'params': {'hidden_units': [256, 256], 'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None, 'use_dense': False}}, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'use_gpu': 0, 'variance_type': 'max', 'algorithm': 'ProbabilisticEnsembleDynamics', 'obs

Epoch 1/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-02 22:23:11 [info     ] ProbabilisticEnsembleDynamics_20231002222259: epoch=1 step=714 epoch=1 metrics={'time_sample_batch': 0.00025343327295212516, 'time_algorithm_update': 0.014632226038379829, 'loss': -32.423491892193546, 'time_step': 0.014955992124327758, 'observation_error': 0.0250744053835917, 'reward_error': 0.0004974876921980015, 'variance': 0.023248634875645952} step=714
2023-10-02 22:23:11 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002222259/model_714.pt


Epoch 2/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-02 22:23:23 [info     ] ProbabilisticEnsembleDynamics_20231002222259: epoch=2 step=1428 epoch=2 metrics={'time_sample_batch': 0.00026366657235709224, 'time_algorithm_update': 0.014832751757624437, 'loss': -39.16295184608267, 'time_step': 0.015169731375216102, 'observation_error': 0.014658510301083456, 'reward_error': 6.89559697864491e-05, 'variance': 0.008658544076944894} step=1428
2023-10-02 22:23:23 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002222259/model_1428.pt


...

Epoch 99/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-02 22:40:30 [info     ] ProbabilisticEnsembleDynamics_20231002222259: epoch=99 step=70686 epoch=99 metrics={'time_sample_batch': 0.00020859889289578137, 'time_algorithm_update': 0.012814775234510918, 'loss': -79.79947139836159, 'time_step': 0.013081943621488512, 'observation_error': 0.015158814820186783, 'reward_error': 2.2440786606136927e-06, 'variance': 0.0069588452347616875} step=70686
2023-10-02 22:40:30 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002222259/model_70686.pt


Epoch 100/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-02 22:40:41 [info     ] ProbabilisticEnsembleDynamics_20231002222259: epoch=100 step=71400 epoch=100 metrics={'time_sample_batch': 0.00020658435607824672, 'time_algorithm_update': 0.012640309267017353, 'loss': -80.09595226373325, 'time_step': 0.01290584779253193, 'observation_error': 0.01638476880753239, 'reward_error': 2.342732445057707e-06, 'variance': 0.006173238159619211} step=71400
2023-10-02 22:40:41 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002222259/model_71400.pt


[(1,
  {'time_sample_batch': 0.00025343327295212516,
   'time_algorithm_update': 0.014632226038379829,
   'loss': -32.423491892193546,
   'time_step': 0.014955992124327758,
   'observation_error': 0.0250744053835917,
   'reward_error': 0.0004974876921980015,
   'variance': 0.023248634875645952}),
 (2,
  {'time_sample_batch': 0.00026366657235709224,
   'time_algorithm_update': 0.014832751757624437,
   'loss': -39.16295184608267,
   'time_step': 0.015169731375216102,
   'observation_error': 0.014658510301083456,
   'reward_error': 6.89559697864491e-05,
   'variance': 0.008658544076944894}),
 (3,
  {'time_sample_batch': 0.00023559963001924401,
   'time_algorithm_update': 0.013485601636208072,
   'loss': -45.77786203058494,
   'time_step': 0.013787621543520973,
   'observation_error': 0.012042198700714673,
   'reward_error': 1.629846289681898e-05,
   'variance': 0.0048035796677338635}),
 (4,
  {'time_sample_batch': 0.00023266080380821762,
   'time_algorithm_update': 0.013436230290837648,
 

## Load Dynamics

In [7]:
# load trained dynamics model
dynamics_model_path = "d3rlpy_logs/ProbabilisticEnsembleDynamics_20231001112902"
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics.from_json(dynamics_model_path + '/params.json')
dynamics.load_model(dynamics_model_path + '/model_53200.pt')

2023-10-01 11:45:52 [warning  ] Unused arguments are passed.   algorithm=ProbabilisticEnsembleDynamics gamma=1.0 generated_maxlen=100000 n_steps=1 real_ratio=1.0


## Train Offline RL Algorithm

In [20]:
encoders = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# give COMBO as the generator argument.
combo = COMBO(dynamics=dynamics, critic_encoder_factory=encoders, actor_encoder_factory=encoders,
              use_gpu=use_gpu)

In [21]:
combo.fit(dataset = train_episodes, eval_episodes=test_episodes, n_steps=100000, n_steps_per_epoch=1000, tensorboard_dir="tensorboard_logs",
         scorers={
            'environment': d3rlpy.metrics.scorer.evaluate_on_environment(eval_env)
        })

2023-10-01 16:30:40 [debug    ] RandomIterator is selected.
2023-10-01 16:30:40 [info     ] Directory is created at d3rlpy_logs/COMBO_20231001163040
2023-10-01 16:30:40 [debug    ] Building models...
2023-10-01 16:30:40 [debug    ] Models have been built.
2023-10-01 16:30:40 [info     ] Parameters are saved to d3rlpy_logs/COMBO_20231001163040/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'conservative_weight': 1.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'gamma':

Epoch 1/100:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-01 16:30:42 [debug    ] 250000 transitions are generated. fake_transitions=250000 real_transitions=78012
2023-10-01 16:31:17 [info     ] COMBO_20231001163040: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.001380002737045288, 'time_algorithm_update': 0.03201468873023987, 'critic_loss': 7.931922992706299, 'actor_loss': -3.1501990002393723, 'temp_loss': 1.4433157570362092, 'temp': 0.9536788986921311, 'time_step': 0.033513587713241574, 'environment': 77.3} step=1000
2023-10-01 16:31:17 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20231001163040/model_1000.pt


Epoch 2/100:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-01 16:31:19 [debug    ] 250000 transitions are generated. fake_transitions=500000 real_transitions=78012
2023-10-01 16:31:53 [info     ] COMBO_20231001163040: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.001382411241531372, 'time_algorithm_update': 0.03167251563072205, 'critic_loss': 7.901574543476105, 'actor_loss': -7.110955627441406, 'temp_loss': 1.2367205929756164, 'temp': 0.868801161646843, 'time_step': 0.033171422481536865, 'environment': 65.6} step=2000
2023-10-01 16:31:53 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20231001163040/model_2000.pt


...

Epoch 99/100:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-01 17:44:38 [debug    ] 250000 transitions are generated. fake_transitions=1250000 real_transitions=78012
2023-10-01 17:45:22 [info     ] COMBO_20231001163040: epoch=99 step=99000 epoch=99 metrics={'time_sample_batch': 0.0013295726776123046, 'time_algorithm_update': 0.029362735509872437, 'critic_loss': 12.186915818214416, 'actor_loss': -32.85153619003296, 'temp_loss': -1.2615893807378598e-05, 'temp': 0.02696601510234177, 'time_step': 0.03081584620475769, 'environment': 1000.0} step=99000
2023-10-01 17:45:22 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20231001163040/model_99000.pt


Epoch 100/100:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-01 17:45:25 [debug    ] 250000 transitions are generated. fake_transitions=1250000 real_transitions=78012
2023-10-01 17:46:11 [info     ] COMBO_20231001163040: epoch=100 step=100000 epoch=100 metrics={'time_sample_batch': 0.0013297867774963378, 'time_algorithm_update': 0.029213391304016113, 'critic_loss': 12.268741159915924, 'actor_loss': -32.98735144805908, 'temp_loss': 0.00033988956674875227, 'temp': 0.026612394601106645, 'time_step': 0.030665696382522582, 'environment': 1000.0} step=100000
2023-10-01 17:46:11 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20231001163040/model_100000.pt


[(1,
  {'time_sample_batch': 0.001380002737045288,
   'time_algorithm_update': 0.03201468873023987,
   'critic_loss': 7.931922992706299,
   'actor_loss': -3.1501990002393723,
   'temp_loss': 1.4433157570362092,
   'temp': 0.9536788986921311,
   'time_step': 0.033513587713241574,
   'environment': 77.3}),
 (2,
  {'time_sample_batch': 0.001382411241531372,
   'time_algorithm_update': 0.03167251563072205,
   'critic_loss': 7.901574543476105,
   'actor_loss': -7.110955627441406,
   'temp_loss': 1.2367205929756164,
   'temp': 0.868801161646843,
   'time_step': 0.033171422481536865,
   'environment': 65.6}),
 (3,
  {'time_sample_batch': 0.001391845703125,
   'time_algorithm_update': 0.031803046703338625,
   'critic_loss': 9.072717978477478,
   'actor_loss': -10.79972409248352,
   'temp_loss': 1.0334303538799285,
   'temp': 0.7956417958140374,
   'time_step': 0.03331618118286133,
   'environment': 102.6}),
 (4,
  {'time_sample_batch': 0.0013628950119018554,
   'time_algorithm_update': 0.03192

## Load the Policy

In [9]:
trained_policy = COMBO()
# initialize with dataset
trained_policy.build_with_dataset(dataset)
# Load entire model parameters.
trained_policy.load_model('d3rlpy_logs/COMBO_20230929153035/model_53000.pt')

## See the policy running

In [10]:
scorer = d3rlpy.metrics.scorer.evaluate_on_environment(eval_env, render=True)
mean_episode_return = scorer(trained_policy)